In [1]:
import pandas as pd
import numpy as np

import folium # for plotting in maps

In [2]:
df = pd.read_csv("Map_2020-03-21.csv")

# Dropping metrics and removing redundancies
df_min = df.drop(['metric_name', 'metric_value'], axis=1)
df_min.drop_duplicates(inplace=True)

del df
df_min.head()

,utc_date,time,crisis_name,start_polygon_id,start_polygon_names,start_x,start_y,start_name_stack,end_polygon_id,end_polygon_names,end_x,end_y,end_name_stack,length_km,level,tile_size,country
0,2020-03-21,16:00,india_coronavirus_disease_prevention_map_mar_2...,640271,Kosi,87.278199,27.143309,Kosi,640273,Narayani,84.879715,27.319324,Narayani,244.414316,LEVEL3,11,NP
5,2020-03-21,08:00,india_coronavirus_disease_prevention_map_mar_2...,640272,Mechi,87.844954,27.157742,Mechi,640272,Mechi,87.844954,27.157742,Mechi,0.032227,LEVEL3,11,NP
10,2020-03-21,16:00,india_coronavirus_disease_prevention_map_mar_2...,640269,Janakpur,86.014259,27.263965,Janakpur,640271,Kosi,87.278199,27.143309,Kosi,162.115691,LEVEL3,11,NP
15,2020-03-21,00:00,india_coronavirus_disease_prevention_map_mar_2...,640267,Lumbini,83.438635,27.738827,Lumbini,640266,Gandaki,84.344753,28.317817,Gandaki,35.186454,LEVEL3,11,NP
20,2020-03-21,00:00,india_coronavirus_disease_prevention_map_mar_2...,640270,Bagmati,85.405209,27.840701,Bagmati,640263,Bheri,81.746217,28.570012,Bheri,365.544806,LEVEL3,11,NP


In [3]:
# Extracting rows where start_name_stack and end_name_stack contains 'Tamil Nadu'

df_tn_start = df_min[(df_min['country'] == 'IN') & (df_min['start_name_stack'].str.contains(r"^Tamil"))]
df_tn_end = df_min[(df_min['country'] == 'IN') & (df_min['end_name_stack'].str.contains(r"^Tamil"))]

In [4]:
# create Map instance; default_location -> lat & long for center of India

def generateBaseMap(default_location=[24, 75], default_zoom_start=4):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map_start = generateBaseMap()
base_map_end = generateBaseMap()

In [5]:
# Map showing points where origin is TN

for _, row in df_tn_start.iterrows():
    # Marker for start point
    folium.CircleMarker([row['start_y'], row['start_x']],
                        radius=3,
                        color="#3db7e4", # divvy color
                       ).add_to(base_map_start)
    
    # Marker for end point
    folium.CircleMarker([row['end_y'], row['end_x']],
                        radius=3,
                        color="#e43d3d", # divvy color
                       ).add_to(base_map_start)

    # Line connecting start and end
    folium.PolyLine([[row['start_y'], row['start_x']], 
                     [row['end_y'], row['end_x']]],
                     color='#4f4f4f', weight=1.0, opacity=0.5).add_to(base_map_start)
    
    
base_map_start.save('testTN_Mar21_Start.html')

In [6]:
# Map showing points where destination is TN

for _, row in df_tn_end.iterrows():
    folium.CircleMarker([row['start_y'], row['start_x']],
                        radius=3,
                        color="#3db7e4", # divvy color
                       ).add_to(base_map_end)

    folium.CircleMarker([row['end_y'], row['end_x']],
                        radius=3,
                        color="#e43d3d", # divvy color
                       ).add_to(base_map_end)

    folium.PolyLine([[row['start_y'], row['start_x']], 
                     [row['end_y'], row['end_x']]],
                     color='#4f4f4f', weight=1.0, opacity=0.5).add_to(base_map_end)
    
base_map_end.save('testTN_Mar21_End.html')